<a href="https://colab.research.google.com/github/Polisetty-Cyril/HealthGPT/blob/master/MedAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sentence-transformers faiss-cpu torch transformers pandas scikit-learn sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 57.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from google.colab import files


In [ ]:
dataset = load_dataset("KryptoniteCrown/synthetic-neurology-QA-dataset", split="train")
neurology_qa = [{"question": row["question"], "answer": row["answer"]} for row in dataset]

cardiology_dataset = load_dataset("ilyassacha/cardiology_qa", split="train")
cardiology_qa = [{"question": row["question"], "answer": row["answer"]} for row in cardiology_dataset]

dermatology_dataset = load_dataset("Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning", split="train")
dermatology_qa = [{"question": row["prompt"], "answer": row["response"]} for row in dermatology_dataset]


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

reformatted_data_neurology.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1452 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/14885 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1654 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/929 [00:00<?, ?B/s]

combined_data.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)
cancer_qa = [{"question": row["Question"], "answer": row["Answer"]} for _, row in df.iterrows()]


Saving CancerQA.csv to CancerQA.csv


In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)
display(df)
diabetes_digestive_kidney_qa = [{"question": row["Question"], "answer": row["Answer"]} for _, row in df.iterrows()]

Saving Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv to Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv


,Question,Answer,topic,split
0,What is (are) Kidney Stones in Adults ?,A kidney stone is a solid piece of material th...,Diabetes_Digestive_Kidney,train
1,What is (are) Kidney Stones in Adults ?,The urinary tract is the bodys drainage system...,Diabetes_Digestive_Kidney,train
2,Who is at risk for Kidney Stones in Adults? ?,"Anyone can get a kidney stone, but some people...",Diabetes_Digestive_Kidney,train
3,What causes Kidney Stones in Adults ?,Kidney stones can form when substances in the ...,Diabetes_Digestive_Kidney,train
4,What is (are) Kidney Stones in Adults ?,Four major types of kidney stones can form:\n ...,Diabetes_Digestive_Kidney,train
...,...,...,...,...
1187,What are the treatments for What I need to kno...,"Once you have cirrhosis, nothing can make all ...",Diabetes_Digestive_Kidney,train
1188,What are the treatments for What I need to kno...,"If too much scar tissue forms, your liver coul...",Diabetes_Digestive_Kidney,train
1189,How to prevent What I need to know about Cirrh...,"To prevent cirrhosis,\n \n- see...",Diabetes_Digestive_Kidney,train
1190,What to do for What I need to know about Cirrh...,- Cirrhosis is scarring of the liver. Scar tis...,Diabetes_Digestive_Kidney,train


In [ ]:
device = torch.device('cpu')  # keep CPU-safe for your environment
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device='cpu')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def build_faiss_index(docs, domain_name="", max_docs=5000):
    """Build FAISS index with progress tracking and better logging."""
    if not docs:
        return None, []

    docs_subset = docs[:min(max_docs, len(docs))]

    # Better logging
    print(f"  {domain_name}: Using {len(docs_subset)}/{len(docs)} documents")

    texts = [d["question"] for d in docs_subset]
    embeddings = embedder.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=True,  # ✅ Changed to True to see progress
        batch_size=64,  # ✅ Increased from 32 for faster processing
        device='cpu'
    )

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.astype(np.float32))

    return index, docs_subset


# Build vector databases
print("\n🔄 Building FAISS indexes...")
vector_dbs = {}

neurology_index, neurology_docs = build_faiss_index(neurology_qa, "Neurology")
cardio_index, cardio_docs = build_faiss_index(cardiology_qa, "Cardiology")
derma_index, derma_docs = build_faiss_index(dermatology_qa, "Dermatology")
cancer_index, cancer_docs = build_faiss_index(cancer_qa, "Cancer")
diabetes_digestive_kidney_index, diabetes_digestive_kidney_docs = build_faiss_index(
    diabetes_digestive_kidney_qa,
    "Diabetes-Digestive-Kidney"
)

# Add to vector_dbs dictionary
if neurology_index:
    vector_dbs["Neurology"] = (neurology_index, neurology_docs)
if cardio_index:
    vector_dbs["Cardiology"] = (cardio_index, cardio_docs)
if derma_index:
    vector_dbs["Dermatology"] = (derma_index, derma_docs)
if cancer_index:
    vector_dbs["Cancer"] = (cancer_index, cancer_docs)
if diabetes_digestive_kidney_index:
    vector_dbs["Diabetes-Digestive-Kidney"] = (diabetes_digestive_kidney_index, diabetes_digestive_kidney_docs)

print(f"\n✓ Built indexes for domains: {list(vector_dbs.keys())}")
print(f"✓ Total domains: {len(vector_dbs)}")

# Print statistics for each domain
print("\n📊 Domain Statistics:")
for domain, (idx, docs) in vector_dbs.items():
    print(f"  {domain}: {len(docs)} documents indexed")



🔄 Building FAISS indexes...
  Neurology: Using 1452/1452 documents


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

  Cardiology: Using 5000/14885 documents


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

  Dermatology: Using 1460/1460 documents


...

  Cancer: Using 729/729 documents


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

  Diabetes-Digestive-Kidney: Using 1192/1192 documents


Batches:   0%|          | 0/19 [00:00<?, ?it/s]


✓ Built indexes for domains: ['Neurology', 'Cardiology', 'Dermatology', 'Cancer', 'Diabetes-Digestive-Kidney']
✓ Total domains: 5

📊 Domain Statistics:
  Neurology: 1452 documents indexed
  Cardiology: 5000 documents indexed
  Dermatology: 1460 documents indexed
  Cancer: 729 documents indexed
  Diabetes-Digestive-Kidney: 1192 documents indexed


In [ ]:
class MedicalExpert(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=512, output_dim=384):
        super().__init__()
        # self.network = nn.Sequential(
        #     nn.Linear(input_dim, hidden_dim),
        #     nn.ReLU(),
        #     nn.Dropout(0.1),
        #     nn.Linear(hidden_dim, output_dim)
        # )
        self.network = nn.Sequential(
          nn.Linear(input_dim, hidden_dim),
          nn.Tanh(),
          nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
class GatingNetwork(nn.Module):
    def __init__(self, input_dim=384, num_experts=3, hidden_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, num_experts)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)

In [ ]:
class MedicalMoE(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=512, output_dim=384, num_experts=3, top_k=2):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = min(top_k, num_experts)
        self.experts = nn.ModuleList([MedicalExpert(input_dim, hidden_dim, output_dim) for _ in range(num_experts)])
        self.gating = GatingNetwork(input_dim, num_experts)
        # expert_names can be set later
        self.expert_names = None

    def forward(self, x, return_router_logits=False):
        # x: (batch, input_dim)
        router_logits = self.gating(x)  # (batch, num_experts)
        if return_router_logits:
            return router_logits
        router_probs = F.softmax(router_logits, dim=-1)
        topk_probs, topk_indices = torch.topk(router_probs, self.top_k, dim=1)
        topk_probs = topk_probs / topk_probs.sum(dim=1, keepdim=True)
        # combine expert outputs (note: experts may be untrained)
        batch_size = x.size(0)
        out = torch.zeros(batch_size, x.size(1), device=x.device)
        for i in range(self.top_k):
            expert_idx_col = topk_indices[:, i]
            weights = topk_probs[:, i]
            # compute expert outputs for all batch where that expert is used
            for b in range(batch_size):
                eidx = expert_idx_col[b].item()
                out[b] += weights[b] * self.experts[eidx](x[b].unsqueeze(0)).squeeze(0)
        return out, topk_indices

    def get_expert_names(self, indices):
        # indices: tensor shape (batch, top_k)
        if self.expert_names is None:
            return [f"expert_{i}" for i in indices[0].tolist()]
        return [self.expert_names[idx.item()] for idx in indices[0]]


In [ ]:
# -------------------------
# Prepare supervised training data for router
# -------------------------
import random # Import random module
# Build mapping domain->label
domain_list = sorted(list(vector_dbs.keys()))
domain_to_label = {d: i for i, d in enumerate(domain_list)}
label_to_domain = {i: d for d, i in domain_to_label.items()}
num_classes = len(domain_list)
print(f"✓ Domain mapping: {domain_to_label}")

training_data = []
samples_per_domain = 1000  # Increased from 500 to 1000

for domain, (idx, docs) in vector_dbs.items():
    label = domain_to_label[domain]
    choices = docs.copy()
    random.shuffle(choices)

    # Use more samples if available
    num_samples = min(samples_per_domain, len(choices))
    print(f"  {domain}: Using {num_samples} training samples")

    for doc in choices[:num_samples]:
        training_data.append({"question": doc["question"], "label": label})

print(f"\n✓ Total training samples: {len(training_data)}")

if len(training_data) == 0:
    raise SystemExit("No training data constructed.")

# Split with stratification
train_data, val_data = train_test_split(
    training_data,
    test_size=0.2,
    random_state=42,
    stratify=[d["label"] for d in training_data]
)

✓ Domain mapping: {'Cancer': 0, 'Cardiology': 1, 'Dermatology': 2, 'Diabetes-Digestive-Kidney': 3, 'Neurology': 4}
  Neurology: Using 1000 training samples
  Cardiology: Using 1000 training samples
  Dermatology: Using 1000 training samples
  Cancer: Using 729 training samples
  Diabetes-Digestive-Kidney: Using 1000 training samples

✓ Total training samples: 4729


In [ ]:
# Dataset class
class MedicalQueryDataset(Dataset):
    def __init__(self, data, embedder):
        self.data = data
        self.questions = [d["question"] for d in data]
        self.labels = [d["label"] for d in data]
        # encode once
        self.embeddings = embedder.encode(self.questions, convert_to_numpy=True, show_progress_bar=False, batch_size=32, device='cpu').astype(np.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return {"embedding": torch.from_numpy(self.embeddings[idx]), "label": torch.tensor(self.labels[idx], dtype=torch.long)}

train_dataset = MedicalQueryDataset(train_data, embedder)
val_dataset   = MedicalQueryDataset(val_data, embedder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

Train size: 3783, Val size: 946


In [ ]:
# -------------------------
# Train router (gating network) only — conservative approach
# -------------------------
moe_model = MedicalMoE(input_dim=train_dataset.embeddings.shape[1], num_experts=num_classes, top_k=2)
moe_model.expert_names = domain_list  # map indices to domain names

def train_router(model, train_loader, val_loader, epochs=5, lr=1e-3):
    model.to(device)
    optimizer = torch.optim.Adam(model.gating.parameters(), lr=lr)  # only train gating
    criterion = nn.CrossEntropyLoss()
    best_val = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch in train_loader:
            emb = batch["embedding"].to(device)
            labels = batch["label"].to(device)
            optimizer.zero_grad()
            logits = model(emb, return_router_logits=True)  # (batch, num_experts)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * emb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        train_acc = 100.0 * correct / total
        avg_loss = running_loss / total

        # val
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                emb = batch["embedding"].to(device)
                labels = batch["label"].to(device)
                logits = model(emb, return_router_logits=True)
                preds = logits.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100.0 * val_correct / val_total
        print(f"Epoch {epoch+1}/{epochs} Loss={avg_loss:.4f} TrainAcc={train_acc:.2f}% ValAcc={val_acc:.2f}%")
        if val_acc > best_val:
            best_val = val_acc
            # save gating only (lighter)
            torch.save({"gating_state_dict": model.gating.state_dict(), "num_experts": num_classes}, "router_gating.pt")
            print("  Saved router_gating.pt")
    return model

print("Training router (this may take a few minutes)...")
moe_model = train_router(moe_model, train_loader, val_loader, epochs=5, lr=1e-3)

# keep for inference
trained_moe_model = moe_model
CHECKPOINT_PATH = "best_moe_model.pt"
torch.save({
    'model_state_dict': trained_moe_model.state_dict(),
    'num_experts': num_classes
}, CHECKPOINT_PATH)
print("Saved full model checkpoint (router + experts) to:", CHECKPOINT_PATH)

Training router (this may take a few minutes)...
Epoch 1/5 Loss=0.8128 TrainAcc=85.38% ValAcc=94.71%
  Saved router_gating.pt
Epoch 2/5 Loss=0.1965 TrainAcc=95.03% ValAcc=95.45%
  Saved router_gating.pt
Epoch 3/5 Loss=0.1252 TrainAcc=96.80% ValAcc=96.83%
  Saved router_gating.pt
Epoch 4/5 Loss=0.0922 TrainAcc=97.73% ValAcc=97.15%
  Saved router_gating.pt
Epoch 5/5 Loss=0.0695 TrainAcc=98.18% ValAcc=97.46%
  Saved router_gating.pt
Saved full model checkpoint (router + experts) to: best_moe_model.pt


In [ ]:
moe_model = MedicalMoE(input_dim=train_dataset.embeddings.shape[1], num_experts=num_classes, top_k=2)
moe_model.expert_names = domain_list  # map indices to domain names

def train_router(model, train_loader, val_loader, epochs=10, lr=1e-3):  # Increased from 5 to 10
    """Train router with early stopping for better convergence."""
    model.to(device)
    optimizer = torch.optim.Adam(model.gating.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    best_val = 0.0
    patience = 3  # Early stopping patience
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch in train_loader:
            emb = batch["embedding"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            logits = model(emb, return_router_logits=True)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * emb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100.0 * correct / total
        avg_loss = running_loss / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                emb = batch["embedding"].to(device)
                labels = batch["label"].to(device)
                logits = model(emb, return_router_logits=True)
                preds = logits.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100.0 * val_correct / val_total
        print(f"Epoch {epoch+1}/{epochs} | Loss={avg_loss:.4f} | Train Acc={train_acc:.2f}% | Val Acc={val_acc:.2f}%")

        # Save best model
        if val_acc > best_val:
            best_val = val_acc
            patience_counter = 0
            torch.save({
                "gating_state_dict": model.gating.state_dict(),
                "num_experts": num_classes,
                "val_acc": val_acc
            }, "router_gating.pt")
            print(f"  ✓ New best model saved (Val Acc: {val_acc:.2f}%)")
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= patience:
            print(f"\n⚠️  Early stopping triggered after {epoch+1} epochs (no improvement for {patience} epochs)")
            break

    print(f"\n✓ Best validation accuracy: {best_val:.2f}%")
    return model

print("\n🔄 Training router with expanded dataset...")
moe_model = train_router(moe_model, train_loader, val_loader, epochs=10, lr=1e-3)

# keep for inference
trained_moe_model = moe_model # Assign the trained moe_model to trained_moe_model
CHECKPOINT_PATH = "best_moe_model.pt"
torch.save({
    'model_state_dict': trained_moe_model.state_dict(),
    'num_experts': num_classes
}, CHECKPOINT_PATH)
print("Saved full model checkpoint (router + experts) to:", CHECKPOINT_PATH)


🔄 Training router with expanded dataset...
Epoch 1/10 | Loss=0.8064 | Train Acc=84.22% | Val Acc=95.14%
  ✓ New best model saved (Val Acc: 95.14%)
Epoch 2/10 | Loss=0.1947 | Train Acc=94.90% | Val Acc=96.19%
  ✓ New best model saved (Val Acc: 96.19%)
Epoch 3/10 | Loss=0.1262 | Train Acc=96.72% | Val Acc=96.83%
  ✓ New best model saved (Val Acc: 96.83%)
Epoch 4/10 | Loss=0.0938 | Train Acc=97.54% | Val Acc=97.25%
  ✓ New best model saved (Val Acc: 97.25%)
Epoch 5/10 | Loss=0.0696 | Train Acc=98.20% | Val Acc=97.67%
  ✓ New best model saved (Val Acc: 97.67%)
Epoch 6/10 | Loss=0.0555 | Train Acc=98.81% | Val Acc=98.41%
  ✓ New best model saved (Val Acc: 98.41%)
Epoch 7/10 | Loss=0.0434 | Train Acc=99.10% | Val Acc=97.99%
Epoch 8/10 | Loss=0.0351 | Train Acc=99.37% | Val Acc=98.20%
Epoch 9/10 | Loss=0.0296 | Train Acc=99.39% | Val Acc=98.73%
  ✓ New best model saved (Val Acc: 98.73%)
Epoch 10/10 | Loss=0.0250 | Train Acc=99.71% | Val Acc=98.52%

✓ Best validation accuracy: 98.73%
Saved fu

In [ ]:
try:
    hyde_tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT")
    hyde_model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT")
    hyde_model.eval()
    print("✓ HyDE (BioGPT) loaded")
except:
    hyde_model = None
    print("⚠ HyDE not available")

try:
    reranker_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
    reranker_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
    reranker_model.eval()
    print("✓ Reranker (FLAN-T5) loaded")
except:
    reranker_model = None
    print("⚠ Reranker not available")

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

✓ HyDE (BioGPT) loaded


...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✓ Reranker (FLAN-T5) loaded


In [ ]:
def hyde_hypothesis(query, max_new_tokens=100):
    """
    Generate a clean hypothetical medical answer (HyDE step).
    Ensures the generated text directly addresses the main focus of the question.
    """
    if hyde_model is None:
        return query
    try:
        prompt = f"""
You are a highly knowledgeable medical expert.
Given the following question, generate a concise, evidence-based hypothetical answer that could appear in a clinical textbook.
Focus on:
- Listing key symptoms, diagnostic clues, and treatments if applicable.
- Using factual reasoning, not imagination.
- Staying strictly within the topic domain of the question.

Question: {query}
Answer:
"""

        print("\n[HyDE] Generating hypothetical medical answer...")
        inputs = hyde_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
        outputs = hyde_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.4,
            do_sample=False,
            pad_token_id=hyde_tokenizer.eos_token_id
        )
        return hyde_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    except Exception as e:
        print(f"[HyDE] Error: {e}")
        return query


In [ ]:
# Add these functions in this order, BEFORE llm_rerank:

def keyword_score(query, answer):
    """Calculate keyword overlap score."""
    query_words = set(query.lower().split()) - {'what', 'is', 'the', 'a', 'how', 'why'}
    answer_words = set(answer.lower().split())
    overlap = len(query_words & answer_words)
    return min(1.0, overlap / max(len(query_words), 1))

def extract_from_context(context, query):
    """Extract relevant sentences from retrieved context."""
    sentences = [s.strip() + '.' for s in context.split('.') if len(s.strip()) > 30]
    query_words = set(query.lower().split()) - {'what', 'are', 'how', 'why'}

    scored = [(len(query_words & set(s.lower().split())), s) for s in sentences[:12]]
    scored = [s for s in scored if s[0] > 0]
    scored.sort(reverse=True)

    return ' '.join([s[1] for s in scored[:3]]) if scored else context[:400]


In [ ]:
def keyword_score(query, answer):
    """Fallback keyword matching score for relevance."""
    query_words = set(query.lower().split())
    answer_words = set(answer.lower().split())

    # Remove common stopwords
    stopwords = {'what', 'is', 'the', 'a', 'an', 'are', 'how', 'why', 'when', 'where', 'in', 'on', 'to', 'for'}
    query_words = query_words - stopwords

    overlap = len(query_words & answer_words)
    return min(1.0, overlap / max(len(query_words), 1))

def llm_rerank(query, candidate_answers, candidate_similarities=None):
    """
    Rerank answers using LLM-based scoring + embedding similarity blending.
    """
    if not candidate_answers:
        return []

    # Normalize embedding similarities
    if candidate_similarities is not None and len(candidate_similarities) > 0:
        min_s, max_s = min(candidate_similarities), max(candidate_similarities)
        if max_s > min_s:
            candidate_similarities = [(s - min_s) / (max_s - min_s) for s in candidate_similarities]
        else:
            candidate_similarities = [0.5] * len(candidate_answers)
    else:
        candidate_similarities = [0.5] * len(candidate_answers)

    scored_answers = []

    for i, ans in enumerate(candidate_answers[:5]):
        rerank_score = 0.0
        try:
            prompt = f"Rate from 0 to 10 how well the following answer addresses the question.\n\nQuestion: {query}\nAnswer: {ans}\nScore:"
            inputs = reranker_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)

            with torch.no_grad():
                outputs = reranker_model.generate(**inputs, max_new_tokens=3, num_beams=1, do_sample=False)

            score_text = reranker_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            import re
            numbers = re.findall(r'\d+\.?\d*', score_text)
            if numbers:
                rerank_score = min(1.0, float(numbers[0]) / 10.0)
            else:
                rerank_score = keyword_score(query, ans)
        except Exception:
            rerank_score = keyword_score(query, ans)

        # Combine scores
        embedding_score = candidate_similarities[i]
        final_score = 0.7 * embedding_score + 0.3 * rerank_score

        scored_answers.append({
            "answer": ans,
            "embedding_score": embedding_score,
            "reranker_score": rerank_score,
            "final_score": final_score,
        })

    return sorted(scored_answers, key=lambda x: x["final_score"], reverse=True)


In [ ]:
# Consider switching to a medical-specific model
try:
    generation_tokenizer = AutoTokenizer.from_pretrained("Locutusque/gpt2-large-medical")
    generation_model = AutoModelForCausalLM.from_pretrained("Locutusque/gpt2-large-medical")
    generation_model.eval()
    generation_tokenizer.pad_token = generation_tokenizer.eos_token
    print("✓ Medical GPT-2 loaded")
except:
    # Fallback to standard
    generation_tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
    generation_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")


def text2text_generate_flant5(query, context, max_length=200):
    """Generate medical answers with optimized prompting."""
    if generation_model is None:
        return extract_from_context(context, query)

    # Enhanced prompt with medical context
    prompt = f"""You are a medical information assistant. Use the provided medical context to answer the question accurately and concisely.

Medical Context:
{context[:700]}

Question: {query}

Instructions:
- Provide a clear, direct answer in 2-3 sentences
- Use only information from the context
- Be specific and factual
- Do not ask questions or make assumptions

Answer:"""

    inputs = generation_tokenizer(
        prompt,
        return_tensors="pt",
        max_length=600,
        truncation=True
    )

    with torch.no_grad():
        outputs = generation_model.generate(
            **inputs,
            max_length=max_length,
            min_length=60,
            num_beams=5,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=1.2
        )

    answer = generation_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Validation
    if len(answer) < 40 or '?' in answer[:50]:
        return extract_from_context(context, query)

    return answer



✓ Medical GPT-2 loaded


In [ ]:
def validate_medical_answer(query, answer, confidence):
    """
    Improved validation - less aggressive on blocking good answers.
    """
    # Basic checks
    if len(answer) < 25:
        return False, "Too short"

    # Remove encoding errors
    answer = answer.replace('ï¿½', '').replace('�', '')

    # Check completeness
    if not answer.strip()[-1] in '.!?':
        sentences = answer.split('.')
        if len(sentences) > 1:
            answer = '. '.join(sentences[:-1]) + '.'
        else:
            return False, "Incomplete"

    # Relaxed relevance check - only block if clearly off-topic
    query_core = set(query.lower().split()) - {'what', 'are', 'the', 'is', 'how', 'why', 'when'}
    answer_words = set(answer.lower().split())

    # Must have at least 20% keyword overlap OR be >100 chars with medical terms
    overlap_ratio = len(query_core & answer_words) / max(len(query_core), 1)
    has_medical_content = any(term in answer.lower() for term in ['disease', 'symptoms', 'treatment', 'condition'])

    if overlap_ratio < 0.2 and not (len(answer) > 100 and has_medical_content):
        return False, "Low relevance"

    return True, answer


In [ ]:
# -------------------------
# Retrieval function (Final Version): integrates HyDE + LLM Reranker
# -------------------------
def retrieve_answer_with_moe_fixed(query, k=5, use_hyde=True):
    # 1️⃣ HyDE generation (optional)
    if use_hyde and hyde_model is not None:
        hyde_text = hyde_hypothesis(query)
        orig_emb = embedder.encode([query], convert_to_numpy=True).astype(np.float32)
        hyde_emb = embedder.encode([hyde_text], convert_to_numpy=True).astype(np.float32)
        query_emb = np.mean([orig_emb, hyde_emb], axis=0)
    else:
        query_emb = embedder.encode([query], convert_to_numpy=True).astype(np.float32)

    # 3️⃣ Route query through Mixture of Experts (MoE) model
    with torch.no_grad():
        q_tensor = torch.from_numpy(query_emb).to(device)
        logits = trained_moe_model(q_tensor, return_router_logits=True)  # (1, num_experts)
        probs = F.softmax(logits, dim=-1).cpu().numpy().squeeze(0)

        # Choose top-k domains
        topk = min(2, len(domain_list))
        top_indices = probs.argsort()[::-1][:topk]
        selected_domains = [label_to_domain[int(i)] for i in top_indices]
        selected_probs = [float(probs[int(i)]) for i in top_indices]

    # 4️⃣ Retrieve candidate documents from FAISS for the selected domains
    candidates = []
    for domain in selected_domains:
        if domain not in vector_dbs:
            continue
        idx, docs = vector_dbs[domain]
        D, I = idx.search(query_emb, k)
        for dist, doc_idx in zip(D[0], I[0]):
            if doc_idx < len(docs):
                candidates.append({
                    "answer": docs[doc_idx]["answer"],
                    "domain": domain,
                    "dist": float(dist)
                })

    if not candidates:
        return {
            "query": query,
            "selected_experts": selected_domains,
            "best_answer": "⚠️ No info found.",
            "confidence_score": 0.0,
            "status": "no_candidates"
        }

    # 5️⃣ Extract candidate answers for reranking
    candidate_texts = [c["answer"] for c in candidates]
    candidate_similarities = [1 / (1 + c["dist"]) for c in candidates]

    # 6️⃣ Apply LLM-based reranker (FLAN-T5 or keyword fallback)
    reranked = llm_rerank(query, candidate_texts, candidate_similarities)

    if not reranked:
        conf = 1.0 / (1.0 + candidates[0]["dist"])
    else:
        conf = reranked[0]["final_score"]

    # SAFETY CHECK: Confidence threshold
    CONFIDENCE_THRESHOLD = 0.45  # Anything below this is unreliable

    if conf < CONFIDENCE_THRESHOLD:
        return {
            "query": query,
            "selected_experts": selected_domains,
            "best_answer": "I'm not confident enough to provide a reliable answer to this question. For accurate medical information, please consult a qualified healthcare professional or trusted medical resources.",
            "confidence_score": conf,
            "candidates_count": len(candidates),
            "status": "rejected_low_confidence"
        }

    # Only generate text-to-text answer if confidence is sufficient
    context = " ".join([c["answer"] for c in candidates[:3]])
    generated_answer = text2text_generate_gpt2(query, context)

    # Additional validation on generated answer
    is_valid, validated_answer = validate_medical_answer(query, generated_answer, conf)

    if not is_valid:
        return {
            "query": query,
            "selected_experts": selected_domains,
            "best_answer": "I cannot provide a reliable answer to this question. Please consult a healthcare professional for accurate information.",
            "confidence_score": conf,
            "status": "rejected_validation_failed",
            "validation_reason": validated_answer
        }

    return {
        "query": query,
        "selected_experts": selected_domains,
        "best_answer": validated_answer,
        "confidence_score": conf,
        "candidates_count": len(candidates),
        "status": "success"
    }


# -------------------------
# Quick test
# -------------------------
tests = [
    "What is the nature of Rosacea, its symptoms, and how is it treated?",
    "What role does trust play in the context of anosognosia?",
    "In a patient presenting with flushing, diarrhea, wheezing, and a heart murmur that increases with inspiration, what is the likely diagnosis?",
    "What are the treatments for Urinary Tract Infection In Adults ?",
    "Does consuming alcohol affect my skin? How?",
    "Who is at risk for Oral Cavity and Oropharyngeal Cancer?",
    "When should antidepressant medications be considered in the treatment of dementia?",
    "What kind of physical activities can be done to achieve the target set forth by the AHA/ACC guidelines?"
]

for t in tests:
    r = retrieve_answer_with_moe_fixed(t, k=7, use_hyde=True)
    print("\n---")
    print("Query:", t)
    print("Selected domains:", r["selected_experts"])
    print("Confidence:", round(r["confidence_score"], 3))
    print("Answer:", r["best_answer"].strip().replace("\n", " "))


[HyDE] Generating hypothetical medical answer...

---
Query: What is the nature of Rosacea, its symptoms, and how is it treated?
Selected domains: ['Dermatology', 'Cardiology']
Confidence: 0.79
Answer: Rosacea is an inflammatory skin disease that primarily affects facial skin.   Rosacea is a disorder characterized by redness, inflammation, and scarring of the skin.  The most common form of rosacea is called erythema multiforme, which is caused by the bacteria Propionibacterium acnes.   For more information, visit the National Rosacea Society website.   What are the most common types of eczema?  The most common type of eczematous dermatitis is called atopic dermatitis.  It occurs when the immune system attacks the epidermis, which is the outermost layer of skin.  Atopic dermatitis can affect any part of the body, including the face, hands, feet, and genitals.   Atopic dermatitis is often treated with topical corticosteroids, such as prednisone or prednisolone.   How common is eczema in